In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!wget https://www.dropbox.com/s/zcwlujrtz3izcw8/gender.tgz
!tar xvzf gender.tgz
!ls

--2020-06-01 07:49:30--  https://www.dropbox.com/s/zcwlujrtz3izcw8/gender.tgz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zcwlujrtz3izcw8/gender.tgz [following]
--2020-06-01 07:49:30--  https://www.dropbox.com/s/raw/zcwlujrtz3izcw8/gender.tgz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc04349d41cc0d166427849f54f7.dl.dropboxusercontent.com/cd/0/inline/A4ytNKuwFyCi0uBk52Z4MmbrnfMP7P7yqx3qO28XLUYp-uK6t63MaUy-lW0y6-De9HZEDDu8fru7xzW5i2HzK9jqywtF6t6rLDFxnoPvLQYqRA/file# [following]
--2020-06-01 07:49:30--  https://uc04349d41cc0d166427849f54f7.dl.dropboxusercontent.com/cd/0/inline/A4ytNKuwFyCi0uBk52Z4MmbrnfMP7P7yqx3qO28XLUYp-uK6t63MaUy-lW0y6-De9HZEDDu8fru7xzW5i2HzK9jqywtF6t6rLDFxnoPvLQYqRA/file
Resolving uc04349d41cc

In [3]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.datasets import cifar10
from keras import models
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

# con tensorflow:
with tf.Session(config=config):
    # con keras:
    set_session(tf.Session(config=config))
    


Using TensorFlow backend.


In [4]:
    
# Load 
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')


## Transforms
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

num_classes = 2

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



(10585, 100, 100, 3)
(2648, 100, 100, 3)


In [0]:

## DEFINE A DATA AUGMENTATION GENERATOR

datagen = ImageDataGenerator(
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)

###########################################################
# Now this is necessary due to the feature normalization: #
datagen.fit(x_train)
###########################################################


In [0]:

## Resnet
def convo_block(y, filtros, num_conv, res=0):

    if (res):
        x=y

    s = 2
    for i in range(num_conv):
        
        y=layers.Conv2D(filtros, kernel_size=(3, 3), strides=(s,s), padding='same')(y)
        s = 1
        y=layers.BatchNormalization()(y)
        y=layers.GaussianNoise(0.3)(y)
        
        if (i<num_conv-1):
            y=layers.ReLU()(y)


    if (res):
        
        x=layers.Conv2D(filtros, kernel_size=(1, 1), strides=(2,2),padding='same')(x)
        y=layers.add([x, y])
        y=layers.ReLU()(y)

    else:
        y=layers.MaxPooling2D(pool_size=(2, 2))(y)
        
    return y


def generate_model( num_conv_blocks):
  
    filtros = 32
    n_conv = 2
    input_layer = layers.Input(shape=(100,100,3))
    x = convo_block(input_layer, filtros, n_conv)
    #Convolutional part
    for i in range(num_conv_blocks-1):
      x = convo_block(x, filtros, n_conv, res=1)
      filtros = filtros * 2

    x=layers.Flatten()(x)
    x=layers.Dense(512)(x)
    x=layers.BatchNormalization()(x)
    x=layers.GaussianNoise(0.3)(x)
    x=layers.ReLU()(x)

    x=layers.Dense(2, activation='softmax')(x)
    model = models.Model(inputs=[input_layer], outputs=[x])

    return model
 

In [7]:
    
model = generate_model(4)
model.summary()

## OPTIM AND COMPILE
opt = SGD(lr=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, cooldown=1,
                              patience=10, min_lr=0.005)

## TRAINING with DA and LRA

batch_size = 100
epochs = 300

history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[reduce_lr],
                            verbose=1)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 50, 50, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 50, 50, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
gaussian_noise_1 (GaussianNoise (None, 50, 50, 32)   0           batch_normalization_1[0][0]      
__________

In [0]:
## Resnet
def convo_block(y, filtros, num_conv, res=0):

    if (res):
        x=y

    y=layers.Conv2D(filtros, kernel_size=(3, 3), strides=(2,2), padding='same')(y)
    
    for i in range(num_conv):
        y=layers.Conv2D(filtros, kernel_size=(3, 3), strides=(1,1), padding='same')(y)
        y=layers.BatchNormalization()(y)
        y=layers.GaussianNoise(0.3)(y)
        y=layers.ReLU()(y)

    y=layers.Conv2D(filtros, kernel_size=(3, 3), strides=(1,1), padding='same')(y)
    y=layers.BatchNormalization()(y)
    y=layers.GaussianNoise(0.3)(y)

    if (res):
        
        x=layers.Conv2D(filtros, kernel_size=(1, 1), strides=(2,2),padding='same')(x)
        y=layers.add([x, y])
        y=layers.ReLU()(y)

    else:
        y=layers.MaxPooling2D(pool_size=(2, 2))(y)
        
    return y


def generate_model(num_conv_blocks):
    
    filtros = 8
    n_conv = 2
    input_layer = layers.Input(shape=(100,100,3))
    x = convo_block(input_layer, filtros, n_conv)
    #Convolutional part
    for i in range(num_conv_blocks-1):
      x = convo_block(x, filtros, n_conv, res=1)

    x=layers.Flatten()(x)
    x=layers.Dense(64)(x)
    x=layers.BatchNormalization()(x)
    x=layers.GaussianNoise(0.3)(x)
    x=layers.ReLU()(x)

    x=layers.Dense(2, activation='softmax')(x)
    model = models.Model(inputs=[input_layer], outputs=[x])

    return model
 

In [11]:
    
model = generate_model(2)
model.summary()


## OPTIM AND COMPILE
opt = SGD(lr=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, cooldown=1,
                              patience=10, min_lr=0.005)

## TRAINING with DA and LRA

batch_size = 100
epochs = 500

history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[reduce_lr],
                            verbose=1)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 50, 50, 8)    224         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 50, 50, 8)    584         conv2d_17[0][0]                  
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 50, 50, 8)    32          conv2d_18[0][0]                  
____________________________________________________________________________________________